In [5]:
import math
import random
import numpy as np
import pandas as pd
import geopandas as gpd
import itertools
import string
import pickle
import treelib
from toydown import GeoUnit, ToyDown
import timeit
from functools import partial
from dask.distributed import Client

In [6]:
def create_tree_from_leaves(leaf_dict):
    """ Given a dictionary, where the keys are the names of leaf nodes (labeled by their path)
        and the corresponding value is the associated attribute counts, this function returns
        the list of GeoUnits that defines the corresponding tree.
    """
    nodes = leaf_dict.copy()
    h = len(list(leaf_dict.keys())[0])
    counts = ["TOTPOP", "VAP"]
    n = len(list(leaf_dict.values())[0][counts[0]])
    level_offsets = [3, 4, 10]
    
    for offset in level_offsets:
        level_names = list(set(list(map(lambda s: s[:-offset], leaf_dict.keys()))))
#         level_counts = [np.zeros(n)]*len(level_names)
        for node in level_names:
            nodes[node] = {c: np.array([v[c] for k, v in leaf_dict.items() if k.startswith(node)]).sum(axis=0) for c in counts}
    return [GeoUnit(k, k[:-3], v) if len(k) == 15 else GeoUnit(k, k[:-1], v) if len(k) == 12 else GeoUnit(k, k[:-6], v) if len(k) == 11 else GeoUnit(k, k[:-3], v) for k, v in nodes.items()]

In [7]:
leaves_dallas_county

{481130001001001: {'VAP': array([5, 0, 5, 0, 0, 0, 0, 0]),
  'TOTPOP': array([5, 0, 5, 0, 0, 0, 0, 0])},
 481130001001004: {'VAP': array([86,  4, 82,  0,  0,  0,  0,  0]),
  'TOTPOP': array([99,  4, 95,  0,  0,  0,  0,  0])},
 481130001001006: {'VAP': array([220,  99,  77,  21,   0,  13,   1,   9]),
  'TOTPOP': array([298, 153,  89,  22,   0,  19,   1,  14])},
 481130001001007: {'VAP': array([88, 39, 26, 20,  0,  3,  0,  0]),
  'TOTPOP': array([98, 46, 29, 20,  0,  3,  0,  0])},
 481130001001008: {'VAP': array([36,  0, 36,  0,  0,  0,  0,  0]),
  'TOTPOP': array([36,  0, 36,  0,  0,  0,  0,  0])},
 481130001001009: {'VAP': array([49,  0, 48,  0,  0,  1,  0,  0]),
  'TOTPOP': array([55,  0, 54,  0,  0,  1,  0,  0])},
 481130001001010: {'VAP': array([43,  2, 41,  0,  0,  0,  0,  0]),
  'TOTPOP': array([52,  2, 50,  0,  0,  0,  0,  0])},
 481130001001011: {'VAP': array([83, 10, 73,  0,  0,  0,  0,  0]),
  'TOTPOP': array([100,  10,  89,   0,   0,   0,   0,   1])},
 481130001001012: {'VAP'

In [8]:
leaves_dallas_county = np.load("../data/recon_dallas_blocks_all_pop_vap.p", allow_pickle=True)
geounits_dc = create_tree_from_leaves(leaves_dallas_county)
geounits_dc.reverse()

In [4]:
geounits_dc[0].attributes["VAP"]

array([1713876,  569832,  657889,  370517,    5740,   90319,     660,
          2075,   16844])

In [5]:
len(leaves_dallas_county)

44113

In [6]:
len(list(filter(lambda x: len(x.name) == 12, geounits_dc)))

1669

In [7]:
# tracts = np.load("../data/dallas_county_tracts_all_vap.p", allow_pickle=True)
counties = np.load("../data/texas_counties_all_pop_vap.p", allow_pickle=True)
del counties["48113"] ## delete extra copy of Dallas County

In [8]:
tx_data = {'TOTPOP': np.array([25145561,  9460921, 11397345,  2886825,    80586,   948426,
                               17920,    33980,   319558]),
           'VAP': np.array([18279737,  6143144,  9074684,  2076282,    61856,   716968,
                            12912,    21205,   172686])}

In [9]:
tx = GeoUnit("48", None, tx_data)

In [10]:
county_geounits = [GeoUnit(geoid, "48", attr) for geoid, attr in counties.items()]

In [11]:
county_geounits.insert(0, tx)

In [12]:
model_all = ToyDown(county_geounits + geounits_dc, 5, 0.5, [1/5, 1/5, 1/5, 1/5, 1/5], gurobi=True)

In [18]:
model_all.show()

48
├── 48001
├── 48003
├── 48005
├── 48007
├── 48009
├── 48011
├── 48013
├── 48015
├── 48017
├── 48019
├── 48021
├── 48023
├── 48025
├── 48027
├── 48029
├── 48031
├── 48033
├── 48035
├── 48037
├── 48039
├── 48041
├── 48043
├── 48045
├── 48047
├── 48049
├── 48051
├── 48053
├── 48055
├── 48057
├── 48059
├── 48061
├── 48063
├── 48065
├── 48067
├── 48069
├── 48071
├── 48073
├── 48075
├── 48077
├── 48079
├── 48081
├── 48083
├── 48085
├── 48087
├── 48089
├── 48091
├── 48093
├── 48095
├── 48097
├── 48099
├── 48101
├── 48103
├── 48105
├── 48107
├── 48109
├── 48111
├── 48113
│   ├── 48113000100
│   │   ├── 481130001001
│   │   │   ├── 481130001001000
│   │   │   ├── 481130001001001
│   │   │   ├── 481130001001002
│   │   │   ├── 481130001001003
│   │   │   ├── 481130001001004
│   │   │   ├── 481130001001005
│   │   │   ├── 481130001001006
│   │   │   ├── 481130001001007
│   │   │   ├── 481130001001008
│   │   │   ├── 481130001001009
│   │   │   ├── 481130001001010
│   │   │   ├── 48113000100101

In [11]:
# model_all.noise_tree(model_all.get_node(model_all.root))

In [14]:
non_neg = model_all.noise_and_adjust(verbose=False)

In [15]:
allow_neg = model_all.noise_and_adjust(verbose=False, bounds=None)

In [22]:
non_neg["481130004043004"]

{'TOTPOP': array([3.17106198e+02, 2.98583862e+01, 2.54011360e+02, 1.21012375e+01,
        2.48590444e+00, 1.86493105e+01, 0.00000000e+00, 5.30484418e-13,
        0.00000000e+00]),
 'VAP': array([3.11362797e+02, 2.98583862e+01, 2.48267958e+02, 1.21012375e+01,
        2.48590444e+00, 1.86493105e+01, 0.00000000e+00, 4.43429559e-13,
        0.00000000e+00])}

In [21]:
allow_neg["481130004043004"]

{'TOTPOP': array([ 3.29954564e+02,  3.02963370e+01,  2.41787090e+02,  3.93325336e+01,
        -5.80575497e+00,  1.98725417e+01, -2.55154137e+00, -1.48133264e-01,
         7.17149088e+00]),
 'VAP': array([ 3.29954564e+02,  3.02963370e+01,  2.41787090e+02,  3.93325336e+01,
        -5.80575497e+00,  1.98725417e+01, -2.55154137e+00, -1.48133264e-01,
         7.17149088e+00])}

In [12]:
len(model_all.level())

44366

In [10]:
client = Client(processes=True, threads_per_worker=1, n_workers=2)
client

Client Scheduler: tcp://127.0.0.1:58532 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 17.18 GB


In [11]:
height = 5
epsilons = [0.25, 0.5, 1, 2]
splits = [("equal", [1/5, 1/5, 1/5, 1/5, 1/5]), ("top_heavy", [1/2, 1/4, 1/12, 1/12, 1/12]), 
          ("mid_heavy", [1/12, 1/6, 1/2, 1/6, 1/12]), ("bottom_heavy", [1/12, 1/12, 1/12, 1/4, 1/2])]
n_samps = 20

In [13]:
for eps in epsilons:
    for name, eps_split in splits:
        model_all = ToyDown(county_geounits + geounits_dc, height, eps, eps_split)

        def run_model(i):
            return model_all.noise_and_adjust(verbose=False, gurobi=True)

        print("Starting {} runs with eps {} and {} split".format(n_samps, eps, name), flush=True)
        client.scatter(model_all, broadcast=True)
        
        for j in range(int(n_samps/2)):
            adjusteds = client.map(run_model, range(2))
            to_sav = np.array((client.gather(adjusteds)))
            np.save("irving_results/dallas_county_2010_VAP_eps_{}_{}_split_run_{}.npy".format(eps, name, j), to_sav)
            del to_sav
            del adjusteds
            
        del model_all

Starting 20 runs with eps 0.25 and equal split
Starting 20 runs with eps 0.25 and top_heavy split
Starting 20 runs with eps 0.25 and mid_heavy split
Starting 20 runs with eps 0.25 and bottom_heavy split
Starting 20 runs with eps 0.5 and equal split
Starting 20 runs with eps 0.5 and top_heavy split
Starting 20 runs with eps 0.5 and mid_heavy split
Starting 20 runs with eps 0.5 and bottom_heavy split
Starting 20 runs with eps 1 and equal split
Starting 20 runs with eps 1 and top_heavy split
Starting 20 runs with eps 1 and mid_heavy split
Starting 20 runs with eps 1 and bottom_heavy split
Starting 20 runs with eps 2 and equal split
Starting 20 runs with eps 2 and top_heavy split
Starting 20 runs with eps 2 and mid_heavy split
Starting 20 runs with eps 2 and bottom_heavy split


In [13]:
client.restart()

distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:57884 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 17.18 GB


In [16]:
timeit.timeit(partial(model_dc.noise_and_adjust, verbose=False, gurobi=True), number=1)

229.06443670700003